#第一章 1.3 Jieba分词

##1.3.1 基于N-gram模型的中文分词

###N-gram模型原理及其在分词中的应用

N-gram是一种基于统计的语言模型，假设当前词的出现概率仅依赖于前N-1个词。
在分词中的应用
**二元模型（Bigram）**最常用，计算相邻词的共现概率：
通过统计语料库中词序列频率，选择概率最大的分词组合。例如：
"研究生命"可切分为"研究/生命"（P=0.8）或"研究生/命"（P=0.2），模型会选择前者。

##1.3.2 基于隐马尔可夫模型（HMM）的中文分词

HMM建模过程
定义状态：在分词任务中，状态通常表示词的边界，例如词的开始（B），词的结束（E），词的中间（M）和独立成词的字符（S）。
观测序列：观测序列即为输入的中文句子。
状态转移概率：定义状态之间转移的概率，例如从状态B转移到状态E的概率。
发射概率：定义在特定状态下生成观测（即词或字符）的概率。
初始状态分布：定义序列开始时各个状态的概率。
训练模型：使用带标签的数据集训练模型，估计状态转移概率、发射概率和初始状态分布。
解码：使用维特比算法（Viterbi Algorithm）或其他解码算法找到最有可能的状态序列，从而实现分词。
分词实现方法
在分词实现中，HMM可以按照以下步骤进行：
提取特征：对输入的中文句子进行预处理，提取特征。
建立模型：根据提取的特征建立HMM模型，定义状态、观测、状态转移概率、发射概率和初始状态分布。
训练模型：使用带标签的数据集对模型进行训练，调整模型参数以最大化观测序列的概率。
应用模型：将训练好的模型应用于新的观测序列，使用解码算法找到最有可能的状态序列，从而实现分词。
输出结果：根据解码得到的状态序列，将输入的中文句子进行分词。

##1.3.3 Jieba分词原理与流程

原理
Jieba分词的核心原理是利用统计语言模型来识别文本中的词语。它首先使用一个词典（通常是基于大量文本数据构建的），然后通过统计方法来确定哪些字符序列更有可能是词语。Jieba分词主要基于以下几个步骤：
词典构建：构建一个词典，其中包含了大量的词语及其对应的频率和词性标注。
句子扫描：对输入的句子进行扫描，生成所有可能的词语组合。
动态规划：使用动态规划算法（如维特比算法）来寻找最优的词语切分路径，即最大概率的词语序列。
HMM模型：对于未登录词（不在词典中的词），使用隐马尔可夫模型（HMM）来进行识别。
流程
加载词典：Jieba分词库自带一个词典，但也允许用户加载自定义词典来提高分词的准确性。
获取关键词：利用Jieba的cut()和lcut()函数来获取分词结果。cut()函数生成一个生成器，而lcut()函数生成一个列表。
去除停用词：Jieba允许用户指定停用词，这些词在分词结果中将被忽略。
数据处理：对分词结果进行后处理，如去除停用词、进行词性标注等。
分词模式：Jieba支持三种分词模式——精确模式、全模式和搜索引擎模式。精确模式试图将句子最精确地切开，全模式把句子中所有可能的词语都扫描出来，搜索引擎模式在精确模式的基础上，对长词再次切分，提高召回率。

In [8]:
def forward_max_matching(vocab, sentence):
    """
    基于正向最大匹配算法 (FMM) 的中文分词函数

    参数：
    vocab (list): 词典，包含了所有可能的词汇
    sentence (str): 需要分词的句子

    返回：
    list: 分词结果的列表
    """
    fmmresult = []  # 存储分词结果的列表
    max_len = max([len(item) for item in vocab])  # 获取词典中最长词的长度
    start = 0  # 分词的起始位置

    # 开始遍历句子，直到处理完整个句子
    while start != len(sentence):
        index = start + max_len  # 尝试匹配最大长度的词
        if index > len(sentence):  # 如果索引超出句子长度，修正为句子末尾
            index = len(sentence)

        # 从当前起始位置尝试从最大长度开始逐步缩小词的长度进行匹配
        while index > start:
            current_substr = sentence[start:index]  # 截取当前子串
            # 如果子串在词典中，或者子串长度为1，则认为是一个有效词
            if current_substr in vocab or len(current_substr) == 1:
                fmmresult.append(current_substr)  # 将有效词加入结果列表
                start = index  # 更新起始位置，跳过已处理的部分
                break  # 找到一个有效词后跳出内层循环继续处理下一个子串
            index -= 1  # 如果没有匹配到有效词，缩短子串长度再试

    return fmmresult  # 返回最终的分词结果

In [9]:
def reverse_directional_max_matching(vocab, sentence):
    """
    Reverse Maximum Matching (RMM) 分词算法。
    从句子的末尾开始，尝试从词典中匹配最长的词直到句子被分割。

    Args:
    vocab (list): 词典，包含了所有可能的词汇
    sentence (str): 需要分词的句子

    返回：
    list: 分词后的结果，按顺序返回分词列表
    """
    rmmresult = []  # 存储分词结果
    max_len = max([len(item) for item in vocab])  # 获取词典中最大词的长度
    start = len(sentence)  # 从句子的末尾开始

    while start != 0:  # 直到处理完整个句子
        index = start - max_len  # 尝试从当前位置往前推最大长度的子串
        if index < 0:
            index = 0  # 防止下标越界，调整为从0开始

        while index < start:  # 向前查找直到找到匹配的词
            current_substr = sentence[index:start]  # 截取当前子串

            # 如果当前子串在词典中，或当前子串长度为1（即单个字符）
            if current_substr in vocab or len(current_substr) == 1:
                rmmresult.insert(0, current_substr)  # 匹配成功，插入到结果列表的开头
                start = index  # 更新起始位置，继续向前匹配
                break  # 找到一个词后跳出内层循环
            index += 1  # 如果当前子串没有匹配，向前移动一个字符继续尝试

    return rmmresult  # 返回最终的分词结果

In [10]:
def bi_directional_matching(vocab, sentence):
    # 获取正向和反向最大匹配的分词结果
    res1 = forward_max_matching(vocab, sentence)
    res2 = reverse_directional_max_matching(vocab, sentence)

    len_res1, len_res2 = len(res1), len(res2)  # 保存长度

    # 如果两个结果的长度相同
    if len_res1 == len_res2:
        # 如果两个结果相同，直接返回
        if res1 == res2:
            return res1
        else:
            # 统计每个结果中长度为1的词的数量
            res1_sn = sum(1 for i in res1 if len(i) == 1)
            res2_sn = sum(1 for i in res2 if len(i) == 1)
            # 返回包含较少单字符词的结果
            return res1 if res1_sn < res2_sn else res2
    else:
        # 返回词数较少的结果
        return res1 if len_res1 < len_res2 else res2

In [11]:
vocab = ['我们', '今天', '在', '野生动物园', '玩']
sentence = '我们是今天在野生动物园玩了'

In [12]:
fmm_result = forward_max_matching(vocab, sentence)
rmm_result = reverse_directional_max_matching(vocab, sentence)
print(fmm_result)
print(rmm_result)

['我们', '是', '今天', '在', '野生动物园', '玩', '了']
['我们', '是', '今天', '在', '野生动物园', '玩', '了']


In [13]:
bm_result = bi_directional_matching(vocab, sentence)
print(bm_result)

['我们', '是', '今天', '在', '野生动物园', '玩', '了']
